<a href="https://colab.research.google.com/github/wallybeamm/neural_entity_recognition/blob/feature%2Finit/enitity_recognation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# Load the dataset
import json
 
# Opening JSON file
data = []
for line in open('/content/test_data.jsonl', 'r'):
    line_dict = json.loads(line)
    data.append((line_dict['data'].replace('\n', ' '), line_dict['label']))

print(data[1][1])
print(len(data))



[[371, 375, 'DATE'], [442, 448, 'GPE'], [450, 460, 'GALLERY'], [462, 471, 'GPE'], [473, 492, 'GALLERY'], [494, 500, 'GPE'], [502, 513, 'GALLERY'], [515, 523, 'GPE'], [542, 550, 'GPE'], [552, 565, 'GALLERY'], [567, 573, 'GPE'], [575, 598, 'GALLERY'], [600, 610, 'GPE'], [637, 644, 'GPE'], [659, 678, 'EVENT'], [710, 718, 'GPE'], [732, 743, 'GPE'], [777, 783, 'GPE'], [855, 861, 'GPE'], [1004, 1009, 'GPE'], [1135, 1150, 'EVENT'], [1188, 1203, 'EVENT'], [1206, 1210, 'DATE'], [1257, 1271, 'EVENT'], [1561, 1571, 'GPE'], [1572, 1596, 'DATE'], [1597, 1620, 'MUSEUM'], [1778, 1782, 'DATE'], [1783, 1802, 'DATE'], [1810, 1815, 'GPE'], [1885, 1890, 'GPE'], [1964, 1979, 'GALLERY'], [2150, 2174, 'DATE'], [2365, 2369, 'DATE'], [2435, 2439, 'GPE'], [2568, 2614, 'EVENT'], [2762, 2768, 'DATE'], [2810, 2814, 'DATE'], [2845, 2861, 'DATE'], [2905, 2909, 'DATE'], [3013, 3017, 'DATE'], [3083, 3087, 'GPE'], [3458, 3462, 'DATE'], [3493, 3509, 'DATE'], [3553, 3557, 'DATE'], [3917, 3949, 'EVENT'], [3950, 3959, 'GPE

In [79]:
# Perform standard imports 
import spacy 
nlp = spacy.load('en_core_web_sm')
# Write a function to display basic entity info: 
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
  else: 
    print('No named entities found.')
doc1 = nlp("Apple is looking at buying U.K. startup for $1 billion") 
show_ents(doc1)


Apple - 0 - 5 - ORG - Companies, agencies, institutions, etc.
U.K. - 27 - 31 - GPE - Countries, cities, states
$1 billion - 44 - 54 - MONEY - Monetary values, including unit


In [80]:
model = None
output_dir = "/content/Model"
n_iter = 100

In [81]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')


Created blank 'en' model


In [82]:
import random
from spacy.training.example import Example

# Update the label list
for annotations in data:
    for ent in annotations[1]:
        ner.add_label(ent[2])
        
# Split the data
train_lenght = int(len(data)*0.7)
print(train_lenght)
train_data = data[:train_lenght]
test_data = data[train_lenght:]

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        batches = spacy.util.minibatch(train_data, size=2)
        for batch in batches:
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                annotations = {'entities' : annotations}
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3)
                
        print(losses)

16


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Alexander Calder Alexander Calder was an American ..." with entities "[[79, 83, 'DATE'], [113, 117, 'DATE'], [220, 232, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Alex Katz From Wikipedia, the free encyclopedia Ju..." with entities "[[384, 397, 'DATE'], [418, 428, 'DATE'], [432, 445...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3

{'ner': 18942.379749941727}
{'ner': 4987.051677990385}
{'ner': 4231.18884784066}
{'ner': 3612.530432629578}
{'ner': 3679.8982940223373}
{'ner': 3337.2641832894474}
{'ner': 2718.840627508329}
{'ner': 2553.4849592945875}
{'ner': 2565.813083163014}
{'ner': 2664.6704621839463}
{'ner': 2488.2132983122347}
{'ner': 2574.018283827748}
{'ner': 2414.242709798471}
{'ner': 2153.720857082727}
{'ner': 2111.667574860482}
{'ner': 1882.7062373658175}
{'ner': 1826.2601211516132}
{'ner': 1615.9602421701757}
{'ner': 1673.8158412298649}
{'ner': 1652.721642636236}
{'ner': 1712.1538961253873}
{'ner': 1666.1982496050446}
{'ner': 1441.0146588696787}
{'ner': 1461.526328412811}
{'ner': 1314.8975864020795}
{'ner': 1381.79867635245}
{'ner': 1247.1058233390909}
{'ner': 1616.8097376103467}
{'ner': 1286.3215499655566}
{'ner': 1167.6607401196825}
{'ner': 1186.7778197514656}
{'ner': 1215.3254518484277}
{'ner': 1102.966238788566}
{'ner': 1053.1828173244419}
{'ner': 991.6792623287495}
{'ner': 988.584837562413}
{'ner': 10

In [85]:
# TEST
for text, _ in test_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('27 Jan 2022', 'DATE'), ('Alicja Kwade', 'ARTIST'), ('i8 Grandi gallery', 'GALLERY'), ('Reykjavik', 'GPE'), ('Alicja Kwade', 'ARTIST'), ('i8 Grandi gallery', 'GALLERY'), ('Reykjavik', 'GPE'), ('Alicja Kwade', 'ARTIST'), ('Alicja Kwade', 'ARTIST'), ('2021', 'DATE'), ('i8 Gallery', 'GALLERY'), ('Reykjavik Iceland', 'GPE'), ('Reykjavik', 'GPE'), ('i8 Gallery', 'GALLERY'), ('Nordics', 'GPE'), ('Olafur Eliasson', 'ARTIST'), ('2017', 'DATE'), ('Hrafn Sturluson', 'ARTIST'), ('i8 Gallery', 'GALLERY'), ('Zurich', 'GPE'), ('Reykjavik', 'GPE'), ('i8 Grandi', 'GALLERY'), ('Reykjavik', 'GPE'), ('i8 Gallery', 'GALLERY'), ('Reykjavik', 'GPE'), ('The gallery', 'EVENT'), ('i8 Grandi', 'GALLERY'), ('Sun', 'DATE'), ('Polish', 'GPE'), ('Berlin', 'GPE'), ('Alicja Kwade', 'ARTIST'), ('i8 Gallery', 'GALLERY'), ('Kwade', 'ARTIST'), ('2010', 'DATE'), ('Alicja Kwade', 'ARTIST'), ('Connectivity', 'GPE'), ('2010', 'DATE'), ('i8 Gallery', 'GALLERY'), ('Reykjavik', 'GPE'), ('Arnarson', 'ARTIST'), ('i8 Gr

In [89]:
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' - '+' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
  else: 
    print('No named entities found.')

for text, _ in test_data:
    doc = nlp(text)
    show_ents(doc)

27 Jan 2022 -  - DATE - Absolute or relative dates or periods
Alicja Kwade -  - ARTIST - None
i8 Grandi gallery -  - GALLERY - None
Reykjavik -  - GPE - Countries, cities, states
Alicja Kwade -  - ARTIST - None
i8 Grandi gallery -  - GALLERY - None
Reykjavik -  - GPE - Countries, cities, states
Alicja Kwade -  - ARTIST - None
Alicja Kwade -  - ARTIST - None
2021 -  - DATE - Absolute or relative dates or periods
i8 Gallery -  - GALLERY - None
Reykjavik Iceland -  - GPE - Countries, cities, states
Reykjavik -  - GPE - Countries, cities, states
i8 Gallery -  - GALLERY - None
Nordics -  - GPE - Countries, cities, states
Olafur Eliasson -  - ARTIST - None
2017 -  - DATE - Absolute or relative dates or periods
Hrafn Sturluson -  - ARTIST - None
i8 Gallery -  - GALLERY - None
Zurich -  - GPE - Countries, cities, states
Reykjavik -  - GPE - Countries, cities, states
i8 Grandi -  - GALLERY - None
Reykjavik -  - GPE - Countries, cities, states
i8 Gallery -  - GALLERY - None
Reykjavik -  - GPE - 

In [86]:
if output_dir is not None:
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/Model
